In [4]:
# import torch
import numpy as np
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df(df_path)

In [26]:
from torch import nn

class georgia_0(nn.Module):
    def __init__(self, win_past=20, features=19, dropout_rate=0.33):
        super().__init__()
        self.flat0 = nn.Flatten()

        self.hidden0 = nn.Linear(features*win_past, 128)
        self.act0 = nn.ReLU()
        self.dropout0 = nn.Dropout(dropout_rate)
        
        self.hidden1 = nn.Linear(128, 128)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.hidden2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.output = nn.Linear(128, 1)
    def forward(self, x):
        x = self.flat0(x)
        x = self.dropout0(self.act0(self.hidden0(x)))
        x = self.dropout1(self.act1(self.hidden1(x)))
        x = self.dropout2(self.act2(self.hidden2(x)))
        x = self.output(x)

        return x


In [31]:
import time
import torch
from torch import optim
import torch_ops as ops

x, y = data["x"], data["y"]
v_split = 0.6
v_cutoff = int(len(x)*v_split)
x_t= torch.tensor(x[:v_cutoff], dtype=torch.float32)
x_v = torch.tensor(x[v_cutoff:], dtype=torch.float32)
y_t = torch.tensor(y[:v_cutoff], dtype=torch.float32)
y_v = torch.tensor(y[v_cutoff:], dtype=torch.float32)

win_past, features = x.shape[1], x.shape[2]
epochs, batch_size, lr, l2_decay = 250, 32, 2e-5, 1e-5
model = georgia_0(win_past=win_past, features=features)
loss_fn = nn.L1Loss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

loss = ops.test_loop(x_t, y_t, model, loss_fn, batch_size)
test_loss = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)
print("---------- Epoch 0 ----------")
print(f"loss: {loss:1.4f}, test loss: {test_loss_new:1.4f}")

t0 = time.perf_counter()
for epoch in range(epochs):
    loss = ops.train_loop(x_t, y_t, model, loss_fn, optimizer, batch_size)
    test_loss_new = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)

    if test_loss_new < test_loss or epoch == epochs - 1:
        print(f"---------- Epoch {epoch + 1} ----------")
        print(f"loss: {loss:1.4f}, test loss: {test_loss_new:1.4f}")
    
    improvement = test_loss_new < test_loss
    if improvement:
        test_loss = test_loss_new
        print(f"Checkpoint: test loss = {test_loss:1.4f} <---------------")
        torch.save(model.state_dict(), r"models/hanzo_0")

t1 = time.perf_counter()

print(f"Training done in {t1 - t0:1.2f} seconds.")

---------- Epoch 0 ----------
loss: 0.1842, test loss: 0.2042
---------- Epoch 250 ----------
loss: 0.0513, test loss: 0.1957
Training done in 2.71 seconds.


In [22]:
x_t.dtype

dtype('float64')